# Gender classification using a Deformable ConvNet (Deformable Convolutional Network) combined with a Vision Transformer (ViT)

In [16]:
#import deeplake
import numpy as np

from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, transforms, models
import torchvision.ops
from collections import OrderedDict

from transformers import ViTFeatureExtractor, ViTForImageClassification
import timm
#from mmdet.models.ops import DeformConv2d

#from mmcv.ops import DeformConv2d

In [3]:
from dataprocessing import DataLoaderWrapper

data=DataLoaderWrapper(batch_size=32)
celebA_train,celebA_val,celebA_test=data.initialize_celebA_dataloaders()
adience=data.initialize_adience_dataloaders()

-

Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/adience



|

...

\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/celeb-a-test



-

hub://activeloop/celeb-a-test loaded successfully.



/Users/tulipmajumder/opt/anaconda3/lib/python3.9/site-packages/deeplake/integrations/pytorch/common.py:126: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


In [24]:
class DeformableGenderClassifier(nn.Module):
    def __init__(self,output=2,deformable=True) -> None:
        super().__init__()
        self.deformable=deformable

        self.layers=nn.Sequential(OrderedDict([
            # first convolutional layer
            ('conv1',torchvision.ops.deform_conv2d(3, 96, 7, padding='valid', stride=4)),  # No padding
            ('relu1',nn.ReLU()),
            ('maxpool1',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('lrn1',nn.LocalResponseNorm(size=5, k=2, alpha=10**(-4), beta=0.75)),

            # second convolutional layer
            ('conv2',torchvision.ops.deform_conv2d(96, 256, 5, padding='same')), # Same padding
            ('relu2',nn.ReLU()),
            ('maxpool2',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('lrn2',nn.LocalResponseNorm(size=5, k=2, alpha=10**(-4), beta=0.75)),

            # third convolutional layer
            ('conv3',torchvision.ops.deform_conv2d(256, 384, 3, padding='same')),  # Same padding
            ('relu3',nn.ReLU()),
            ('maxpool3',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('flatten',nn.Flatten()),

        ]))
        
        
        self.fc_layers=nn.Sequential(
            ('fc1', nn.Linear(384 * 6 * 6, 512)),
            ('relu4', nn.ReLU()),
            ('dropout1', nn.Dropout(0.5)),

            ('fc2', nn.Linear(512, 512)),
            ('relu5', nn.ReLU()),
            ('dropout2', nn.Dropout(0.5)),

            ('fc3', nn.Linear(512, output))  # Output = number of classes
        )
        
        self.prob=nn.Softmax(dim=1) # new stuff to check if its causes harm

    def forward(self,x):
        x=self.layers(x)
        x = self.fc_layers(x)
        prob=self.prob(x)
        return prob

In [4]:
'''
class DeformableGenderClassifier(nn.Module):
    def __init__(self, num_classes):
        super(DeformableGenderClassifier, self).__init__()

        # Deformable Convolutional Layers
        self.deform_conv1 = torchvision.ops.DeformConv2d(3, 96, kernel_size=7, padding=0)
        self.deform_conv2 = torchvision.ops.DeformConv2d(96, 256, kernel_size=5, padding=0)
        self.deform_conv3 = torchvision.ops.DeformConv2d(256, 384, kernel_size=3, padding=0)

        # Vision Transformer (ViT)
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        num_features = self.vit.head.in_features
        self.vit.head = nn.Identity()  # Remove the default classification head
        self.classifier = nn.Linear(num_features, num_classes) 

    def forward(self, x):
        # Compute the offset tensors using torchvision utility
        offset1 = torchvision.ops.deform_conv2d_offset(x, self.deform_conv1.weight.size()[2])
        offset2 = torchvision.ops.deform_conv2d_offset(x, self.deform_conv2.weight.size()[2])
        offset3 = torchvision.ops.deform_conv2d_offset(x, self.deform_conv3.weight.size()[2])

        # Apply deformable convolution with the computed offsets
        x = self.deform_conv1(x, offset1)
        x = self.deform_conv2(x, offset2)
        x = self.deform_conv3(x, offset3)
        
        # Pass the output through the Vision Transformer (ViT)
        x = self.vit(x)
        
        # Final classification
        x = self.classifier(x)
        return x


In [25]:
# Create an instance of the DeformableGenderClassifier with the desired number of classes
num_classes = 2  # 2 classes: male and female
model = DeformableGenderClassifier(num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

AttributeError: 'int' object has no attribute 'shape'

In [6]:
model

DeformableGenderClassifier(
  (deform_conv1): DeformConv2d(3, 96, kernel_size=(7, 7), stride=(1, 1))
  (deform_conv2): DeformConv2d(96, 256, kernel_size=(5, 5), stride=(1, 1))
  (deform_conv3): DeformConv2d(256, 384, kernel_size=(3, 3), stride=(1, 1))
  (vit): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()


In [8]:
from sklearn.metrics import accuracy_score

num_epochs = 100

# Lists to store predictions and ground truth labels
gender_preds = []
age_preds = []
gender_gt = []
age_gt = []

# Lists to track losses
gender_losses = []
age_losses = []

for epoch in range(num_epochs):
    model.train()
    for inputs, age_labels, gender_labels in celebA_train:
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Calculate age and gender losses
        age_loss = criterion(outputs, age_labels)  
        gender_loss = criterion(outputs, gender_labels)  

        total_loss = age_loss + gender_loss
        total_loss.backward()
        optimizer.step()

        # Store losses for monitoring
        gender_losses.append(gender_loss.item())
        age_losses.append(age_loss.item())

    # Evaluate the model on the test set periodically (e.g., after each epoch)
    model.eval()
    with torch.no_grad():
        for inputs, age_labels, gender_labels in celebA_test:
            outputs = model(inputs)
            age_preds.extend(outputs['age'].argmax(dim=1).cpu().numpy())
            gender_preds.extend(outputs['gender'].argmax(dim=1).cpu().numpy())
            age_gt.extend(age_labels.cpu().numpy())
            gender_gt.extend(gender_labels.cpu().numpy())

    # Calculate accuracy and print
    gender_accuracy = accuracy_score(gender_gt, gender_preds)
    age_accuracy = accuracy_score(age_gt, age_preds)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Gender Accuracy: {gender_accuracy}, Age Accuracy: {age_accuracy}")


/Users/tulipmajumder/opt/anaconda3/lib/python3.9/site-packages/deeplake/integrations/pytorch/common.py:126: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


AttributeError: module 'torchvision.ops' has no attribute 'deform_conv2d_offset'